In [1]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import catboost

In [2]:
# Loading Data

train = pd.read_csv('data/train.csv')
songs = pd.read_csv('data/songs.csv')
song_labels = pd.read_csv('data/song_labels.csv')
test = pd.read_csv('data/test.csv')
save_for_later = pd.read_csv('data/save_for_later.csv')
dummy_submission = pd.read_csv('data/dummy_submission.csv')

In [3]:
from sklearn.model_selection import train_test_split
train_init, train_rem = train_test_split(train, train_size = 0.3, random_state = 1023)

In [4]:
customer_id_list = train['customer_id'].unique()

In [5]:
song_id_list = train['song_id'].unique()

In [6]:
# Initializing Matrix Factorization
learning_rate = 1e-2
iters = 100
dim = 100
reg = 0.05
# Initialization

customer_weights = {}
for customer in customer_id_list:
    np.random.seed(0)
    customer_weights[customer] = np.random.uniform(0, 10e-10, dim)
    
song_weights = {}
for song in song_id_list:
    np.random.seed(0)
    song_weights[song] = np.random.uniform(0, 10e-10, dim)

In [7]:
# Training Matrix Factorization
import time
start = time.time()
for i in range(iters):
    for k in range(len(train_init.index.to_numpy())):
        customer_weight = customer_weights[train_init['customer_id'].iloc[k]]
        song_weight = song_weights[train_init['song_id'].iloc[k]]
        y = train_init['score'].iloc[k]
        
        temp = y - np.dot(customer_weight, song_weight)
        customer_weight_new = customer_weight + learning_rate*(temp*song_weight-reg*customer_weight)
        song_weight_new = song_weight + learning_rate*(temp*customer_weight-reg*song_weight)
        customer_weights[train_init['customer_id'].iloc[k]] = customer_weight_new
        song_weights[train_init['song_id'].iloc[k]] = song_weight_new
    estimates_train_rem = []
    for k in range(len(train_rem.index.to_numpy())):
        customer_weight = customer_weights[train_rem['customer_id'].iloc[k]]
        song_weight = song_weights[train_rem['song_id'].iloc[k]]
        estimate = np.dot(customer_weight, song_weight)
        estimates_train_rem.append(estimate)
    estimates_train_init = []
    for k in range(len(train_init.index.to_numpy())):
        customer_weight = customer_weights[train_init['customer_id'].iloc[k]]
        song_weight = song_weights[train_init['song_id'].iloc[k]]
        estimate = np.dot(customer_weight, song_weight)
        estimates_train_init.append(estimate)
    y_train_rem = train_rem['score'].to_numpy()
    y_train_init = train_init['score'].to_numpy()
    estimates_train_rem = np.array(estimates_train_rem)
    estimates_train_init = np.array(estimates_train_init)
    train_error_rem = (1/len(y_train_rem))*np.linalg.norm(y_train_rem - estimates_train_rem)**2
    train_error_init = (1/len(y_train_init))*np.linalg.norm(y_train_init - estimates_train_init)**2
    print(f'iter {i} rem {train_error_rem} init {train_error_init}')
print(time.time()-start)

iter 0 rem 16.468380150358996 init 16.471464380938077
iter 1 rem 16.468380150358936 init 16.47146438093789
iter 2 rem 16.468380150347283 init 16.471464380924125
iter 3 rem 16.468380149892933 init 16.47146438038872
iter 4 rem 16.468380132103764 init 16.471464359401303
iter 5 rem 16.468379435037072 init 16.471463536816636
iter 6 rem 16.468352117963374 init 16.471431298626765
iter 7 rem 16.467281812686732 init 16.47016818293299
iter 8 rem 16.425717185030987 init 16.421159841086634
iter 9 rem 15.215597728893067 init 15.031569498465272
iter 10 rem 10.298923720012684 init 9.898934327811594
iter 11 rem 6.494676186801549 init 6.062209058167402
iter 12 rem 4.24886096224336 init 3.79021818874653
iter 13 rem 2.9652660706698746 init 2.508350074538458
iter 14 rem 2.226793946611942 init 1.7945647046929702
iter 15 rem 1.784314524793565 init 1.3851894630010986
iter 16 rem 1.5061857374770444 init 1.140014436723588
iter 17 rem 1.3242825711015878 init 0.9876494127092059
iter 18 rem 1.201515364501376 init

In [8]:
customer_weights_df = pd.DataFrame(customer_weights)

In [9]:
customer_weights_df = customer_weights_df.transpose()

In [ ]:
customer_weights_df['customer_id'] = customer_weights_df.index

In [ ]:
train_rem = train_rem.merge(customer_weights_df, on = 'customer_id', how = 'left')

In [ ]:
song_weights_df = pd.DataFrame(song_weights)

In [ ]:
song_weights_df = song_weights_df.transpose()

In [ ]:
song_weights_df['song_id'] = song_weights_df.index

In [ ]:
songs = songs.merge(song_weights_df, on = 'song_id', how = 'left')

In [ ]:
estimates_train = []
for k in range(len(train_rem.index.to_numpy())):
    customer_weight = customer_weights[train_rem['customer_id'].iloc[k]]
    song_weight = song_weights[train_rem['song_id'].iloc[k]]
    estimate = np.dot(customer_weight, song_weight)
    estimates_train.append(estimate)

In [ ]:
train_rem['estimates'] = estimates_train

In [ ]:
song_labels_new = song_labels.groupby('label_id').sum().sort_values('count', ascending = False)

In [ ]:
song_labels_pivot = song_labels.pivot_table(index = 'platform_id', columns = 'label_id', values = 'count')

In [ ]:
song_labels_pivot = song_labels_pivot.fillna(0)

In [ ]:
song_labels_pivot = song_labels_pivot.applymap(lambda x: np.log(1+np.abs(x)))

In [ ]:
song_labels_pivot

In [ ]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=100, max_iter = 1000, verbose = 1, random_state = 1)

In [ ]:
song_labels_transformed = nmf.fit_transform(song_labels_pivot)

In [ ]:
song_labels_transformed.shape

In [ ]:
song_labels_transformed_df = pd.DataFrame(song_labels_transformed, index = song_labels_pivot.index)

In [ ]:
song_labels_transformed_df

In [ ]:
songs = pd.merge(songs, song_labels_transformed_df, on = 'platform_id', how = 'left')

In [ ]:
songs

In [ ]:
songs = songs.drop(['platform_id'], axis = 1)

In [ ]:
song_scores = train.merge(train.groupby('song_id').mean(), on = 'song_id', how = 'left')[['song_id', 'score_y']].drop_duplicates('song_id', keep = 'first')

In [ ]:
songs = songs.merge(song_scores, on = 'song_id', how = 'left')

In [ ]:
song_num_ratings = train['song_id'].value_counts().to_frame()

In [ ]:
song_num_ratings['num_ratings'] = song_num_ratings['song_id']

In [ ]:
song_num_ratings['song_id'] = song_num_ratings.index

In [ ]:
songs = songs.merge(song_num_ratings, on = 'song_id', how = 'left')

In [ ]:
songs.drop_duplicates('song_id', keep = 'first', inplace = True)

In [ ]:
f = pd.merge(train_rem, save_for_later, on=['customer_id','song_id'], how='left', indicator='Exist')

In [ ]:
train_rem = f

In [ ]:
X_train = pd.merge(train_rem, songs, on = ['song_id'], how = 'left')

In [ ]:
Y_train = X_train['score']

In [ ]:
X_train.drop(['score'], axis = 1, inplace = True)

In [ ]:
X_train['released_year'] = X_train['released_year'].fillna(-999)
X_train['language'] = X_train['language'].fillna('none')
X_train['number_of_comments'] = X_train['number_of_comments'].fillna(-999)

In [ ]:
X_train.columns.to_numpy().tolist().index('Exist')

In [10]:
from catboost import CatBoostRegressor

In [11]:
model = CatBoostRegressor(cat_features = [0,1, 103, 105])

In [ ]:
model.fit(X_train, Y_train, cat_features = [0,1, 103, 105], plot = True)

In [ ]:
test = test.merge(customer_weights_df, on = 'customer_id', how = 'left')

In [ ]:
estimates_test = []
for k in range(len(test.index.to_numpy())):
    customer_weight = customer_weights[test['customer_id'].iloc[k]]
    song_weight = song_weights[test['song_id'].iloc[k]]
    estimate = np.dot(customer_weight, song_weight)
    estimates_test.append(estimate)

In [ ]:
test['estimates'] = estimates_test

In [ ]:
test = pd.merge(test, save_for_later, on=['customer_id','song_id'], how='left', indicator='Exist')

In [ ]:
X_test = pd.merge(test, songs, on = ['song_id'], how = 'left')

In [ ]:
X_test['released_year'] = X_test['released_year'].fillna(-999)
X_test['language'] = X_test['language'].fillna('none')
X_test['number_of_comments'] = X_test['number_of_comments'].fillna(-999)

In [ ]:
y_test_pred = model.predict(X_test)

In [ ]:
y_final = pd.DataFrame(y_test_pred)

In [ ]:
y_final['score'] = y_final[0]

In [ ]:
y_final.drop(0, axis = 1, inplace = True)

In [ ]:
y_final['test_row_id'] = y_final.index

In [ ]:
y_final = y_final[['test_row_id', 'score']]

In [ ]:
y_final.to_csv('colab_nmfv2.csv', index = False)